In [2]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.retrievers.multi_query import MultiQueryRetriever

In [6]:
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [19]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
)

chat_model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.5
)

vector_store = Chroma.from_documents(
    documents=all_docs,
    embedding=embedding_model,
)

In [20]:
similarity_retriever = vector_store.as_retriever(
    search_kwargs={"k": 5},
    search_type="similarity",
)

In [25]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    llm=chat_model,
    retriever=vector_store.as_retriever(
        search_kwargs={"k": 5},
    ),
)

In [26]:
query = "How to improve energy levels and maintain balance?"

similarity_results = similarity_retriever.invoke(query)
multiquery_results = multiquery_retriever.invoke(query)

for i,doc in enumerate(similarity_results):
    print(f"Similarity Result {i+1}: {doc.page_content}")
print("\n")

for i,doc in enumerate(multiquery_results):
    print(f"MultiQuery Result {i+1}: {doc.page_content}")
print("\n")

Similarity Result 1: Drinking sufficient water throughout the day helps maintain metabolism and energy.
Similarity Result 2: The solar energy system in modern homes helps balance electricity demand.
Similarity Result 3: Consuming leafy greens and fruits helps detox the body and improve longevity.
Similarity Result 4: Mindfulness and controlled breathing lower cortisol and improve mental clarity.
Similarity Result 5: Photosynthesis enables plants to produce energy by converting sunlight.


MultiQuery Result 1: Drinking sufficient water throughout the day helps maintain metabolism and energy.
MultiQuery Result 2: The solar energy system in modern homes helps balance electricity demand.
MultiQuery Result 3: Photosynthesis enables plants to produce energy by converting sunlight.
MultiQuery Result 4: Regular walking boosts heart health and can reduce symptoms of depression.
MultiQuery Result 5: Consuming leafy greens and fruits helps detox the body and improve longevity.
MultiQuery Result 6